# Advent of code 2023

## [Day 4](https://adventofcode.com/2023/day/4): Scratchcards

### Solution in python

#### part 1

In [ ]:
file = open('input.txt', 'r')
part1 = 0
for ix, line in enumerate(file):
    card, line = line.split(":")
    card = int(card.split(" ")[-1])
    winning, had =  [[int(num.strip()) for num in l.split(" ") if num] for l in line.split("|")]
    winning_had = [x for x in had if x in winning]
    score = 0 if not winning_had else 2**(len(winning_had)-1)
    part1 += score
part1

#### part 2

In [ ]:
file = open('input.txt', 'r')
scratch_cards = {}
for ix, line in enumerate(file):
    card, line = line.split(":")
    card = int(card.split(" ")[-1])
    winning, had =  [[int(num.strip()) for num in l.split(" ") if num] for l in line.split("|")]
    winning_had = [x for x in had if x in winning]
    matches = len(winning_had)
    scratch_cards[card] = {"matches": matches, "num": 1}

In [ ]:
for card_num in scratch_cards:
    for offset in range(1, scratch_cards[card_num]['matches'] + 1):
        scratch_cards[card_num + offset]['num'] += scratch_cards[card_num]['num']

part2 = 0
for card_num in scratch_cards:
    part2 += scratch_cards[card_num]['num']
part2